## **Code : Result Ver.3(300W, 18kW)**
- Writer : Donghyeon Kim
- Update : 2022.11.16.
- 대상 : 통합 데이터(태양광 사용 300W, 18kW & 미사용)

<hr>

## **Prior Settings**

In [1]:
# 라이브러리 설정
import os
import pandas as pd
import numpy as np
import openpyxl
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 사용자 함수 호출
# get_project_root : 파일의 상위-상위 경로 호출
# get_name_root : 모든 사용자 이름 호출
# get_name_use_final : 태양광 사용자 이름(최종) / 단독주택이라는 공통점 있음.
# get_name_not_final : 태양광 미사용자 이름(최종)
# get_name_use_final_3kw : 태양광 사용자 이름(최종) 중 3kW 사용자
from pack_utils import get_project_root, get_name_root, get_name_use_final, get_name_not_final, get_name_use_final_3kw

<hr>

## **Data : 300W**
### 1) Hour(1시간 단위)

In [3]:
# All Data
root = get_project_root()
folder_root = os.path.join(root, 'data_final')
file_name = os.path.join(folder_root, 'final_data_hour.xlsx')
rawdata = pd.read_excel(file_name)

# 300W 한정
df_all_hour = rawdata[rawdata['설비용량(kW)'].isin(['300W', np.nan])]

# Filtering
# (1) Using Solar
df_all_hour_use = df_all_hour[df_all_hour.type == 'use']

# (2) Not Using Solar
df_all_hour_not = df_all_hour[df_all_hour.type == 'not']

In [4]:
df_all_hour.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175249 entries, 39047 to 263702
Data columns (total 25 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   가구번호                    175249 non-null  int64  
 1   연도                      175249 non-null  int64  
 2   월                       175249 non-null  int64  
 3   일                       175249 non-null  int64  
 4   시간                      175249 non-null  int64  
 5   설비용량(kW)                9093 non-null    object 
 6   발전량(kWh)                9093 non-null    float64
 7   전력소비량(kWh)              175249 non-null  float64
 8   수전전력량(kWh)              175249 non-null  float64
 9   잉여전력량(kWh)              9093 non-null    float64
 10  잉여전력량/발전량               9093 non-null    float64
 11  자가소비율                   9093 non-null    float64
 12  자가공급률                   9093 non-null    float64
 13  temperature             175249 non-null  float64
 14  uws_10m         

### 2) Day(1일 단위)

In [5]:
# All Data
root = get_project_root()
folder_root = os.path.join(root, 'data_final')
file_name2 = os.path.join(folder_root, 'final_data_day.xlsx')
rawdata2 = pd.read_excel(file_name2)

# 300W 한정
df_all_day = rawdata2[rawdata2['설비용량(kW)'].isin(['300W', np.nan])]

# Filtering
# (1) Using Solar
df_all_day_use = df_all_day[df_all_day.type == 'use']

# (2) Not Using Solar
df_all_day_not = df_all_day[df_all_day.type == 'not']

In [6]:
df_all_day.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7583 entries, 1681 to 11762
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   가구번호        7583 non-null   int64  
 1   연도          7583 non-null   int64  
 2   월           7583 non-null   int64  
 3   일           7583 non-null   int64  
 4   설비용량(kW)    381 non-null    object 
 5   발전량(kWh)    381 non-null    float64
 6   발전시간        381 non-null    float64
 7   이용률         381 non-null    float64
 8   전력소비량(kWh)  7583 non-null   float64
 9   수전전력량(kWh)  7583 non-null   float64
 10  잉여전력량(kWh)  381 non-null    float64
 11  잉여전력량/발전량   381 non-null    float64
 12  자가소비율       381 non-null    float64
 13  자가공급률       381 non-null    float64
 14  type        7583 non-null   object 
 15  owner       7583 non-null   object 
 16  ym          7583 non-null   object 
dtypes: float64(9), int64(4), object(4)
memory usage: 1.0+ MB


<hr>

## **<평균>**

### **1. 월간/계절별/1년 가구당 평균 전력 소비량**
- 태양광 사용 가구
- 태양광 미사용 가구

In [7]:
# Result Dictionary
result = {}
result['use'] = []
result['not'] = []

# Fill Dictionary
date_list = ['2021/5', '2021/6', '2021/7', '2021/8', '2021/9', '2021/10', '2021/11', '2021/12',
             '2022/1', '2022/2', '2022/3', '2022/4']

for i in range(len(date_list)):
    # 1) Year/Month 필터링
    df_all_day_use_f = df_all_day_use[df_all_day_use.ym == date_list[i]]
    df_all_day_not_f = df_all_day_not[df_all_day_not.ym == date_list[i]]
    
    # 2) 태양광 사용/미사용 가구별 값 합산
    use_consum = np.sum(df_all_day_use_f['전력소비량(kWh)'])
    not_consum = np.sum(df_all_day_not_f['전력소비량(kWh)'])
    
    # 3) 평균 구하기
    use_consum_mean = round(use_consum / len(df_all_day_use_f.owner.unique()), 1)
    not_consum_mean = round(not_consum / len(df_all_day_not_f.owner.unique()), 1)
    
    # 4) 값 넣기
    result['use'].append(use_consum_mean)
    result['not'].append(not_consum_mean)
    
result = pd.DataFrame(result)
result.index = date_list
result

,use,not
2021/5,407.1,238.2
2021/6,394.1,232.0
2021/7,562.5,336.2
2021/8,495.1,320.2
2021/9,377.0,267.8
2021/10,409.2,242.8
2021/11,392.4,257.7
2021/12,399.8,291.7
2022/1,435.2,309.9
2022/2,372.3,292.7


In [8]:
# Result Dictionary
result = {}
result['use'] = []
result['not'] = []

# 1) 계절 필터링
df_all_day_use_spring = df_all_day_use[(df_all_day_use.ym == '2021/5') | (df_all_day_use.ym == '2022/3') | (df_all_day_use.ym == '2022/4')]
df_all_day_not_spring = df_all_day_not[(df_all_day_not.ym == '2021/5') | (df_all_day_not.ym == '2022/3') | (df_all_day_not.ym == '2022/4')]
    
df_all_day_use_summer = df_all_day_use[(df_all_day_use.ym == '2021/6') | (df_all_day_use.ym == '2021/7') | (df_all_day_use.ym == '2021/8')]
df_all_day_not_summer = df_all_day_not[(df_all_day_not.ym == '2021/6') | (df_all_day_not.ym == '2021/7') | (df_all_day_not.ym == '2021/8')]
    
df_all_day_use_autumn = df_all_day_use[(df_all_day_use.ym == '2021/9') | (df_all_day_use.ym == '2021/10') | (df_all_day_use.ym == '2021/11')]
df_all_day_not_autumn = df_all_day_not[(df_all_day_not.ym == '2021/9') | (df_all_day_not.ym == '2021/10') | (df_all_day_not.ym == '2021/11')]
    
df_all_day_use_winter = df_all_day_use[(df_all_day_use.ym == '2021/12') | (df_all_day_use.ym == '2022/1') | (df_all_day_use.ym == '2022/2')]
df_all_day_not_winter = df_all_day_not[(df_all_day_not.ym == '2021/12') | (df_all_day_not.ym == '2022/1') | (df_all_day_not.ym == '2022/2')]
    
# 2) 태양광 사용/미사용 가구별 값 합산
use_consum_spring = np.sum(df_all_day_use_spring['전력소비량(kWh)'])
not_consum_spring = np.sum(df_all_day_not_spring['전력소비량(kWh)'])
    
use_consum_summer = np.sum(df_all_day_use_summer['전력소비량(kWh)'])
not_consum_summer = np.sum(df_all_day_not_summer['전력소비량(kWh)'])
    
use_consum_autumn = np.sum(df_all_day_use_autumn['전력소비량(kWh)'])
not_consum_autumn = np.sum(df_all_day_not_autumn['전력소비량(kWh)'])
    
use_consum_winter = np.sum(df_all_day_use_winter['전력소비량(kWh)'])
not_consum_winter = np.sum(df_all_day_not_winter['전력소비량(kWh)'])
    
# 3) 평균 구하기
use_consum_spring_mean = round(use_consum_spring / (len(df_all_day_use_spring.owner.unique())*3), 1)
not_consum_spring_mean = round(not_consum_spring / (len(df_all_day_not_spring.owner.unique())*3), 1)

use_consum_summer_mean = round(use_consum_summer / (len(df_all_day_use_summer.owner.unique())*3), 1)
not_consum_summer_mean = round(not_consum_summer / (len(df_all_day_not_summer.owner.unique())*3), 1)
    
use_consum_autumn_mean = round(use_consum_autumn / (len(df_all_day_use_autumn.owner.unique())*3), 1)
not_consum_autumn_mean = round(not_consum_autumn / (len(df_all_day_not_autumn.owner.unique())*3), 1)
    
use_consum_winter_mean = round(use_consum_winter / (len(df_all_day_use_winter.owner.unique())*3), 1)
not_consum_winter_mean = round(not_consum_winter / (len(df_all_day_not_winter.owner.unique())*3), 1)
    
# 4) 값 넣기
result['use'] = [use_consum_spring_mean, use_consum_summer_mean, use_consum_autumn_mean, use_consum_winter_mean]
result['not'] = [not_consum_spring_mean, not_consum_summer_mean, not_consum_autumn_mean, not_consum_winter_mean]

result = pd.DataFrame(result)
result.index = ['spring', 'summer', 'autumn', 'winter']
result

,use,not
spring,366.0,239.2
summer,483.9,296.1
autumn,392.9,256.1
winter,402.5,298.1


In [9]:
# Result Dictionary
result = {}
result['use'] = []
result['not'] = []

# 1) 1년 필터링
df_all_day_use_year = df_all_day_use[(df_all_day_use.ym == '2021/5') | (df_all_day_use.ym == '2021/6') | (df_all_day_use.ym == '2021/7') |
                                    (df_all_day_use.ym == '2021/8') | (df_all_day_use.ym == '2021/9') | (df_all_day_use.ym == '2021/10') |
                                    (df_all_day_use.ym == '2021/11') | (df_all_day_use.ym == '2021/12') | (df_all_day_use.ym == '2022/1') |
                                    (df_all_day_use.ym == '2022/2') | (df_all_day_use.ym == '2022/3') | (df_all_day_use.ym == '2022/4')]
df_all_day_not_year = df_all_day_not[(df_all_day_not.ym == '2021/5') | (df_all_day_not.ym == '2021/6') | (df_all_day_not.ym == '2021/7') |
                                    (df_all_day_not.ym == '2021/8') | (df_all_day_not.ym == '2021/9') | (df_all_day_not.ym == '2021/10') |
                                    (df_all_day_not.ym == '2021/11') | (df_all_day_not.ym == '2021/12') | (df_all_day_not.ym == '2022/1') |
                                    (df_all_day_not.ym == '2022/2') | (df_all_day_not.ym == '2022/3') | (df_all_day_not.ym == '2022/4')]
    
# 2) 태양광 사용/미사용 가구별 값 합산
use_consum_year = np.sum(df_all_day_use_year['전력소비량(kWh)'])
not_consum_year = np.sum(df_all_day_not_year['전력소비량(kWh)'])
    
# 3) 평균 구하기
use_consum_year_mean = round(use_consum_year / (len(df_all_day_use_year.owner.unique())*12), 1)
not_consum_year_mean = round(not_consum_year / (len(df_all_day_not_year.owner.unique())*12), 1)
    
# 4) 값 넣기
result['use'] = use_consum_year_mean
result['not'] = not_consum_year_mean

result

{'use': 411.3, 'not': 272.4}

<hr>

### **2. 월간/계절별/1년 평균 가구당 전력 생산량(발전량)**
- 태양광 사용 가구

In [10]:
# Result Dictionary
result = {}
result['발전량(kWh)'] = []

# Fill Dictionary
date_list = ['2021/5', '2021/6', '2021/7', '2021/8', '2021/9', '2021/10', '2021/11', '2021/12',
             '2022/1', '2022/2', '2022/3', '2022/4']

for i in range(len(date_list)):
    # 1) Year/Month 필터링
    df_all_day_use_f = df_all_day_use[df_all_day_use.ym == date_list[i]]
    
    # 2) 태양광 사용/미사용 가구별 값 합산
    use_consum = np.sum(df_all_day_use_f['발전량(kWh)'])
    
    # 3) 평균 구하기
    use_consum_mean = round(use_consum / len(df_all_day_use_f.owner.unique()), 1)

    # 4) 값 넣기
    result['발전량(kWh)'].append(use_consum_mean)
    
result = pd.DataFrame(result)
result.index = date_list
result

,발전량(kWh)
2021/5,33.6
2021/6,31.1
2021/7,30.2
2021/8,28.3
2021/9,27.4
2021/10,21.2
2021/11,16.3
2021/12,16.9
2022/1,19.7
2022/2,22.8


In [11]:
# Result Dictionary
result = {}
result['발전량(kWh)'] = []

# 1) 계절 필터링
df_all_day_use_spring = df_all_day_use[(df_all_day_use.ym == '2021/5') | (df_all_day_use.ym == '2022/3') | (df_all_day_use.ym == '2022/4')]
df_all_day_use_summer = df_all_day_use[(df_all_day_use.ym == '2021/6') | (df_all_day_use.ym == '2021/7') | (df_all_day_use.ym == '2021/8')]
df_all_day_use_autumn = df_all_day_use[(df_all_day_use.ym == '2021/9') | (df_all_day_use.ym == '2021/10') | (df_all_day_use.ym == '2021/11')]
df_all_day_use_winter = df_all_day_use[(df_all_day_use.ym == '2021/12') | (df_all_day_use.ym == '2022/1') | (df_all_day_use.ym == '2022/2')]

# 2) 태양광 사용/미사용 가구별 값 합산
use_consum_spring = np.sum(df_all_day_use_spring['발전량(kWh)'])
use_consum_summer = np.sum(df_all_day_use_summer['발전량(kWh)'])
use_consum_autumn = np.sum(df_all_day_use_autumn['발전량(kWh)'])
use_consum_winter = np.sum(df_all_day_use_winter['발전량(kWh)'])
    
# 3) 평균 구하기
use_consum_spring_mean = round(use_consum_spring / (len(df_all_day_use_spring.owner.unique())*3), 1)
use_consum_summer_mean = round(use_consum_summer / (len(df_all_day_use_summer.owner.unique())*3), 1)
use_consum_autumn_mean = round(use_consum_autumn / (len(df_all_day_use_autumn.owner.unique())*3), 1)
use_consum_winter_mean = round(use_consum_winter / (len(df_all_day_use_winter.owner.unique())*3), 1)
    
# 4) 값 넣기
result['발전량(kWh)'] = [use_consum_spring_mean, use_consum_summer_mean, use_consum_autumn_mean, use_consum_winter_mean]

result = pd.DataFrame(result)
result.index = ['spring', 'summer', 'autumn', 'winter']
result

,발전량(kWh)
spring,30.3
summer,29.9
autumn,21.6
winter,19.8


In [12]:
# Result Dictionary
result = {}
result['발전량(kWh)'] = []

# 1) 1년 필터링
df_all_day_use_year = df_all_day_use[(df_all_day_use.ym == '2021/5') | (df_all_day_use.ym == '2021/6') | (df_all_day_use.ym == '2021/7') |
                                    (df_all_day_use.ym == '2021/8') | (df_all_day_use.ym == '2021/9') | (df_all_day_use.ym == '2021/10') |
                                    (df_all_day_use.ym == '2021/11') | (df_all_day_use.ym == '2021/12') | (df_all_day_use.ym == '2022/1') |
                                    (df_all_day_use.ym == '2022/2') | (df_all_day_use.ym == '2022/3') | (df_all_day_use.ym == '2022/4')]

# 2) 태양광 사용/미사용 가구별 값 합산
use_consum_year = np.sum(df_all_day_use_year['발전량(kWh)'])

# 3) 평균 구하기
use_consum_year_mean = round(use_consum_year / (len(df_all_day_use_year.owner.unique())*12), 1)

# 4) 값 넣기
result['발전량(kWh)'] = use_consum_year_mean

result

{'발전량(kWh)': 25.4}

<hr>

### **3. 월간/계절별/1년 평균 가구당 전력 수출량(잉여전력량)**
- 태양광 사용 가구

In [13]:
# Result Dictionary
result = {}
result['잉여전력량(kWh)'] = []
result['잉여전력량/발전량'] = []

# Fill Dictionary
date_list = ['2021/5', '2021/6', '2021/7', '2021/8', '2021/9', '2021/10', '2021/11', '2021/12',
             '2022/1', '2022/2', '2022/3', '2022/4']

for i in range(len(date_list)):
    # 1) Year/Month 필터링
    df_all_day_use_f = df_all_day_use[df_all_day_use.ym == date_list[i]]
    
    # 2) 태양광 사용/미사용 가구별 값 합산
    use_export = np.sum(df_all_day_use_f['잉여전력량(kWh)'])
    use_percent = np.sum(df_all_day_use_f['잉여전력량/발전량'])
    
    # 3) 평균 구하기
    use_export_mean = round(use_export / len(df_all_day_use_f.owner.unique()), 1)
    use_percent_mean = round(use_percent / len(df_all_day_use_f.owner.unique()), 1)

    # 4) 값 넣기
    result['잉여전력량(kWh)'].append(use_export_mean)
    result['잉여전력량/발전량'].append(use_percent_mean)
    
result = pd.DataFrame(result)
result.index = date_list
result

,잉여전력량(kWh),잉여전력량/발전량
2021/5,0.9,0.6
2021/6,0.4,0.3
2021/7,0.2,0.2
2021/8,0.4,0.3
2021/9,0.9,0.8
2021/10,0.5,0.5
2021/11,0.5,0.8
2021/12,0.4,0.5
2022/1,0.7,0.9
2022/2,0.9,1.0


In [14]:
# Result Dictionary
result = {}
result['잉여전력량(kWh)'] = []
result['잉여전력량/발전량'] = []

# 1) 계절 필터링
df_all_day_use_spring = df_all_day_use[(df_all_day_use.ym == '2021/5') | (df_all_day_use.ym == '2022/3') | (df_all_day_use.ym == '2022/4')]
df_all_day_use_summer = df_all_day_use[(df_all_day_use.ym == '2021/6') | (df_all_day_use.ym == '2021/7') | (df_all_day_use.ym == '2021/8')]
df_all_day_use_autumn = df_all_day_use[(df_all_day_use.ym == '2021/9') | (df_all_day_use.ym == '2021/10') | (df_all_day_use.ym == '2021/11')]
df_all_day_use_winter = df_all_day_use[(df_all_day_use.ym == '2021/12') | (df_all_day_use.ym == '2022/1') | (df_all_day_use.ym == '2022/2')]

# 2) 태양광 사용/미사용 가구별 값 합산
use_consum_spring = np.sum(df_all_day_use_spring['잉여전력량(kWh)'])
use_percent_spring = np.sum(df_all_day_use_spring['잉여전력량/발전량'])

use_consum_summer = np.sum(df_all_day_use_summer['잉여전력량(kWh)'])
use_percent_summer = np.sum(df_all_day_use_summer['잉여전력량/발전량'])

use_consum_autumn = np.sum(df_all_day_use_autumn['잉여전력량(kWh)'])
use_percent_autumn = np.sum(df_all_day_use_autumn['잉여전력량/발전량'])

use_consum_winter = np.sum(df_all_day_use_winter['잉여전력량(kWh)'])
use_percent_winter = np.sum(df_all_day_use_winter['잉여전력량/발전량'])
    
# 3) 평균 구하기
use_consum_spring_mean = round(use_consum_spring / (len(df_all_day_use_spring.owner.unique())*3), 1)
use_percent_spring_mean = round(use_percent_spring / (len(df_all_day_use_spring.owner.unique())*3), 1)

use_consum_summer_mean = round(use_consum_summer / (len(df_all_day_use_summer.owner.unique())*3), 1)
use_percent_summer_mean = round(use_percent_summer / (len(df_all_day_use_summer.owner.unique())*3), 1)

use_consum_autumn_mean = round(use_consum_autumn / (len(df_all_day_use_autumn.owner.unique())*3), 1)
use_percent_autumng_mean = round(use_percent_autumn / (len(df_all_day_use_autumn.owner.unique())*3), 1)

use_consum_winter_mean = round(use_consum_winter / (len(df_all_day_use_winter.owner.unique())*3), 1)
use_percent_winter_mean = round(use_percent_winter / (len(df_all_day_use_winter.owner.unique())*3), 1)

# 4) 값 넣기
result['잉여전력량(kWh)'] = [use_consum_spring_mean, use_consum_summer_mean, use_consum_autumn_mean, use_consum_winter_mean]
result['잉여전력량/발전량'] = [use_percent_spring_mean, use_percent_summer_mean, use_percent_autumng_mean, use_percent_winter_mean]

result = pd.DataFrame(result)
result.index = ['spring', 'summer', 'autumn', 'winter']
result

,잉여전력량(kWh),잉여전력량/발전량
spring,1.1,0.8
summer,0.4,0.3
autumn,0.6,0.7
winter,0.7,0.8


In [15]:
# Result Dictionary
result = {}
result['잉여전력량(kWh)'] = []
result['잉여전력량/발전량'] = []

# 1) 1년 필터링
df_all_day_use_year = df_all_day_use[(df_all_day_use.ym == '2021/5') | (df_all_day_use.ym == '2021/6') | (df_all_day_use.ym == '2021/7') |
                                    (df_all_day_use.ym == '2021/8') | (df_all_day_use.ym == '2021/9') | (df_all_day_use.ym == '2021/10') |
                                    (df_all_day_use.ym == '2021/11') | (df_all_day_use.ym == '2021/12') | (df_all_day_use.ym == '2022/1') |
                                    (df_all_day_use.ym == '2022/2') | (df_all_day_use.ym == '2022/3') | (df_all_day_use.ym == '2022/4')]

# 2) 태양광 사용/미사용 가구별 값 합산
use_consum_year = np.sum(df_all_day_use_year['잉여전력량(kWh)'])
use_percent_year = np.sum(df_all_day_use_year['잉여전력량/발전량'])

# 3) 평균 구하기
use_consum_year_mean = round(use_consum_year / (len(df_all_day_use_year.owner.unique())*12), 1)
use_percent_year_mean = round(use_percent_year / (len(df_all_day_use_year.owner.unique())*12), 1)

# 4) 값 넣기
result['잉여전력량(kWh)'] = use_consum_year_mean
result['잉여전력량/발전량'] = use_percent_year_mean

result

{'잉여전력량(kWh)': 0.7, '잉여전력량/발전량': 0.7}

<hr>

### **4. SSR, SCR, 설비이용률, 발전시간**
- 태양광 사용 가구

In [16]:
# Result Dictionary
result = {}
result['SSR'] = []
result['SCR'] = []
result['설비이용률'] = []
result['발전시간'] = []

# Fill Dictionary
date_list = ['2021/5', '2021/6', '2021/7', '2021/8', '2021/9', '2021/10', '2021/11', '2021/12',
             '2022/1', '2022/2', '2022/3', '2022/4']

for i in range(len(date_list)):
    # 1) Year/Month 필터링
    df_all_day_use_f = df_all_day_use[df_all_day_use.ym == date_list[i]]
    
    # 2) 태양광 사용/미사용 가구별 값 합산
    ssr = np.sum(df_all_day_use_f['자가공급률'])
    scr = np.sum(df_all_day_use_f['자가소비율'])
    use_rate = np.sum(df_all_day_use_f['이용률'])
    use_hour = np.sum(df_all_day_use_f['발전시간'])
    
    # 3) 평균 구하기
    ssr_mean = round(ssr / len(df_all_day_use_f), 2)
    scr_mean = round(scr / len(df_all_day_use_f), 2)
    use_rate_mean = round(use_rate / len(df_all_day_use_f), 2)
    use_hour_mean = round(use_hour / len(df_all_day_use_f), 2)
    
    # 4) 값 넣기
    result['SSR'].append(ssr_mean)
    result['SCR'].append(scr_mean)
    result['설비이용률'].append(use_rate_mean)
    result['발전시간'].append(use_hour_mean)
    
result = pd.DataFrame(result)
result.index = date_list
result

,SSR,SCR,설비이용률,발전시간
2021/5,0.08,0.98,15.05,3.61
2021/6,0.08,0.99,14.41,3.46
2021/7,0.06,0.96,13.98,3.35
2021/8,0.06,0.99,12.68,3.04
2021/9,0.07,0.97,12.71,3.05
2021/10,0.05,0.98,9.50,2.28
2021/11,0.04,0.97,7.54,1.81
2021/12,0.05,0.98,7.56,1.82
2022/1,0.05,0.97,8.85,2.12
2022/2,0.06,0.96,11.30,2.71


<hr>

### **5. 월간/계절별/1년 평균 가구당 전력 그리드 소비량**
- 태양광 사용 가구
- 태양광 미사용 가구

In [17]:
# Result Dictionary
result = {}
result['use'] = []
result['not'] = []

# Fill Dictionary
date_list = ['2021/5', '2021/6', '2021/7', '2021/8', '2021/9', '2021/10', '2021/11', '2021/12',
             '2022/1', '2022/2', '2022/3', '2022/4']

for i in range(len(date_list)):
    # 1) Year/Month 필터링
    df_all_day_use_f = df_all_day_use[df_all_day_use.ym == date_list[i]]
    df_all_day_not_f = df_all_day_not[df_all_day_not.ym == date_list[i]]
    
    # 2) 태양광 사용/미사용 가구별 값 합산
    use_consum = np.sum(df_all_day_use_f['수전전력량(kWh)'])
    not_consum = np.sum(df_all_day_not_f['수전전력량(kWh)'])
    
    # 3) 평균 구하기
    use_consum_mean = round(use_consum / len(df_all_day_use_f.owner.unique()), 1)
    not_consum_mean = round(not_consum / len(df_all_day_not_f.owner.unique()), 1)
    
    # 4) 값 넣기
    result['use'].append(use_consum_mean)
    result['not'].append(not_consum_mean)
    
result = pd.DataFrame(result)
result.index = date_list
result

,use,not
2021/5,374.4,238.2
2021/6,363.5,232.0
2021/7,532.5,336.2
2021/8,467.3,320.2
2021/9,350.5,267.8
2021/10,388.5,242.8
2021/11,376.6,257.7
2021/12,383.3,291.7
2022/1,416.2,309.9
2022/2,350.4,292.7


In [18]:
# Result Dictionary
result = {}
result['use'] = []
result['not'] = []

# 1) 계절 필터링
df_all_day_use_spring = df_all_day_use[(df_all_day_use.ym == '2021/5') | (df_all_day_use.ym == '2022/3') | (df_all_day_use.ym == '2022/4')]
df_all_day_not_spring = df_all_day_not[(df_all_day_not.ym == '2021/5') | (df_all_day_not.ym == '2022/3') | (df_all_day_not.ym == '2022/4')]
    
df_all_day_use_summer = df_all_day_use[(df_all_day_use.ym == '2021/6') | (df_all_day_use.ym == '2021/7') | (df_all_day_use.ym == '2021/8')]
df_all_day_not_summer = df_all_day_not[(df_all_day_not.ym == '2021/6') | (df_all_day_not.ym == '2021/7') | (df_all_day_not.ym == '2021/8')]
    
df_all_day_use_autumn = df_all_day_use[(df_all_day_use.ym == '2021/9') | (df_all_day_use.ym == '2021/10') | (df_all_day_use.ym == '2021/11')]
df_all_day_not_autumn = df_all_day_not[(df_all_day_not.ym == '2021/9') | (df_all_day_not.ym == '2021/10') | (df_all_day_not.ym == '2021/11')]
    
df_all_day_use_winter = df_all_day_use[(df_all_day_use.ym == '2021/12') | (df_all_day_use.ym == '2022/1') | (df_all_day_use.ym == '2022/2')]
df_all_day_not_winter = df_all_day_not[(df_all_day_not.ym == '2021/12') | (df_all_day_not.ym == '2022/1') | (df_all_day_not.ym == '2022/2')]
    
# 2) 태양광 사용/미사용 가구별 값 합산
use_consum_spring = np.sum(df_all_day_use_spring['수전전력량(kWh)'])
not_consum_spring = np.sum(df_all_day_not_spring['수전전력량(kWh)'])
    
use_consum_summer = np.sum(df_all_day_use_summer['수전전력량(kWh)'])
not_consum_summer = np.sum(df_all_day_not_summer['수전전력량(kWh)'])
    
use_consum_autumn = np.sum(df_all_day_use_autumn['수전전력량(kWh)'])
not_consum_autumn = np.sum(df_all_day_not_autumn['수전전력량(kWh)'])
    
use_consum_winter = np.sum(df_all_day_use_winter['수전전력량(kWh)'])
not_consum_winter = np.sum(df_all_day_not_winter['수전전력량(kWh)'])
    
# 3) 평균 구하기
use_consum_spring_mean = round(use_consum_spring / (len(df_all_day_use_spring.owner.unique())*3), 1)
not_consum_spring_mean = round(not_consum_spring / (len(df_all_day_not_spring.owner.unique())*3), 1)

use_consum_summer_mean = round(use_consum_summer / (len(df_all_day_use_summer.owner.unique())*3), 1)
not_consum_summer_mean = round(not_consum_summer / (len(df_all_day_not_summer.owner.unique())*3), 1)
    
use_consum_autumn_mean = round(use_consum_autumn / (len(df_all_day_use_autumn.owner.unique())*3), 1)
not_consum_autumn_mean = round(not_consum_autumn / (len(df_all_day_not_autumn.owner.unique())*3), 1)
    
use_consum_winter_mean = round(use_consum_winter / (len(df_all_day_use_winter.owner.unique())*3), 1)
not_consum_winter_mean = round(not_consum_winter / (len(df_all_day_not_winter.owner.unique())*3), 1)
    
# 4) 값 넣기
result['use'] = [use_consum_spring_mean, use_consum_summer_mean, use_consum_autumn_mean, use_consum_winter_mean]
result['not'] = [not_consum_spring_mean, not_consum_summer_mean, not_consum_autumn_mean, not_consum_winter_mean]

result = pd.DataFrame(result)
result.index = ['spring', 'summer', 'autumn', 'winter']
result

,use,not
spring,336.8,239.2
summer,454.4,296.1
autumn,371.8,256.1
winter,383.3,298.1


In [19]:
# Result Dictionary
result = {}
result['use'] = []
result['not'] = []

# 1) 1년 필터링
df_all_day_use_year = df_all_day_use[(df_all_day_use.ym == '2021/5') | (df_all_day_use.ym == '2021/6') | (df_all_day_use.ym == '2021/7') |
                                    (df_all_day_use.ym == '2021/8') | (df_all_day_use.ym == '2021/9') | (df_all_day_use.ym == '2021/10') |
                                    (df_all_day_use.ym == '2021/11') | (df_all_day_use.ym == '2021/12') | (df_all_day_use.ym == '2022/1') |
                                    (df_all_day_use.ym == '2022/2') | (df_all_day_use.ym == '2022/3') | (df_all_day_use.ym == '2022/4')]
df_all_day_not_year = df_all_day_not[(df_all_day_not.ym == '2021/5') | (df_all_day_not.ym == '2021/6') | (df_all_day_not.ym == '2021/7') |
                                    (df_all_day_not.ym == '2021/8') | (df_all_day_not.ym == '2021/9') | (df_all_day_not.ym == '2021/10') |
                                    (df_all_day_not.ym == '2021/11') | (df_all_day_not.ym == '2021/12') | (df_all_day_not.ym == '2022/1') |
                                    (df_all_day_not.ym == '2022/2') | (df_all_day_not.ym == '2022/3') | (df_all_day_not.ym == '2022/4')]
    
# 2) 태양광 사용/미사용 가구별 값 합산
use_consum_year = np.sum(df_all_day_use_year['수전전력량(kWh)'])
not_consum_year = np.sum(df_all_day_not_year['수전전력량(kWh)'])
    
# 3) 평균 구하기
use_consum_year_mean = round(use_consum_year / (len(df_all_day_use_year.owner.unique())*12), 1)
not_consum_year_mean = round(not_consum_year / (len(df_all_day_not_year.owner.unique())*12), 1)
    
# 4) 값 넣기
result['use'] = use_consum_year_mean
result['not'] = not_consum_year_mean

result

{'use': 386.6, 'not': 272.4}

<hr>

### **6. SCR(자가소비율)**
- 태양광 사용 가구
- 개인정보 포함으로 코드 삭제함.

<hr>

### **7. SSR(자가공급률)**
- 태양광 사용 가구
- 개인정보 포함으로 코드 삭제함.

<hr>

<hr>

## **Data : 18kW**
### 1) Hour(1시간 단위)

In [22]:
# All Data
root = get_project_root()
folder_root = os.path.join(root, 'data_final')
file_name = os.path.join(folder_root, 'final_data_hour.xlsx')
rawdata = pd.read_excel(file_name)

# 18kW 한정
df_all_hour = rawdata[rawdata['설비용량(kW)'].isin(['18kW', np.nan])]

# Filtering
# (1) Using Solar
df_all_hour_use = df_all_hour[df_all_hour.type == 'use']

# (2) Not Using Solar
df_all_hour_not = df_all_hour[df_all_hour.type == 'not']

In [23]:
df_all_hour.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 176334 entries, 87369 to 263702
Data columns (total 25 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   가구번호                    176334 non-null  int64  
 1   연도                      176334 non-null  int64  
 2   월                       176334 non-null  int64  
 3   일                       176334 non-null  int64  
 4   시간                      176334 non-null  int64  
 5   설비용량(kW)                10178 non-null   object 
 6   발전량(kWh)                10178 non-null   float64
 7   전력소비량(kWh)              176334 non-null  float64
 8   수전전력량(kWh)              176334 non-null  float64
 9   잉여전력량(kWh)              10178 non-null   float64
 10  잉여전력량/발전량               10178 non-null   float64
 11  자가소비율                   10178 non-null   float64
 12  자가공급률                   10178 non-null   float64
 13  temperature             176334 non-null  float64
 14  uws_10m         

### 2) Day(1일 단위)

In [24]:
# All Data
root = get_project_root()
folder_root = os.path.join(root, 'data_final')
file_name2 = os.path.join(folder_root, 'final_data_day.xlsx')
rawdata2 = pd.read_excel(file_name2)

# 18kW 한정
df_all_day = rawdata2[rawdata2['설비용량(kW)'].isin(['18kW', np.nan])]

# Filtering
# (1) Using Solar
df_all_day_use = df_all_day[df_all_day.type == 'use']

# (2) Not Using Solar
df_all_day_not = df_all_day[df_all_day.type == 'not']

In [25]:
df_all_day.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7628 entries, 3724 to 11762
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   가구번호        7628 non-null   int64  
 1   연도          7628 non-null   int64  
 2   월           7628 non-null   int64  
 3   일           7628 non-null   int64  
 4   설비용량(kW)    426 non-null    object 
 5   발전량(kWh)    426 non-null    float64
 6   발전시간        426 non-null    float64
 7   이용률         426 non-null    float64
 8   전력소비량(kWh)  7628 non-null   float64
 9   수전전력량(kWh)  7628 non-null   float64
 10  잉여전력량(kWh)  426 non-null    float64
 11  잉여전력량/발전량   426 non-null    float64
 12  자가소비율       426 non-null    float64
 13  자가공급률       426 non-null    float64
 14  type        7628 non-null   object 
 15  owner       7628 non-null   object 
 16  ym          7628 non-null   object 
dtypes: float64(9), int64(4), object(4)
memory usage: 1.0+ MB


<hr>

## **<평균>**

### **1. 월간/계절별/1년 가구당 평균 전력 소비량**
- 태양광 사용 가구
- 태양광 미사용 가구

In [26]:
# Result Dictionary
result = {}
result['use'] = []
result['not'] = []

# Fill Dictionary
date_list = ['2021/5', '2021/6', '2021/7', '2021/8', '2021/9', '2021/10', '2021/11', '2021/12',
             '2022/1', '2022/2', '2022/3', '2022/4']

for i in range(len(date_list)):
    # 1) Year/Month 필터링
    df_all_day_use_f = df_all_day_use[df_all_day_use.ym == date_list[i]]
    df_all_day_not_f = df_all_day_not[df_all_day_not.ym == date_list[i]]
    
    # 2) 태양광 사용/미사용 가구별 값 합산
    use_consum = np.sum(df_all_day_use_f['전력소비량(kWh)'])
    not_consum = np.sum(df_all_day_not_f['전력소비량(kWh)'])
    
    # 3) 평균 구하기
    use_consum_mean = round(use_consum / len(df_all_day_use_f.owner.unique()), 1)
    not_consum_mean = round(not_consum / len(df_all_day_not_f.owner.unique()), 1)
    
    # 4) 값 넣기
    result['use'].append(use_consum_mean)
    result['not'].append(not_consum_mean)
    
result = pd.DataFrame(result)
result.index = date_list
result

,use,not
2021/5,206.7,238.2
2021/6,183.6,232.0
2021/7,284.6,336.2
2021/8,258.5,320.2
2021/9,172.5,267.8
2021/10,180.6,242.8
2021/11,171.6,257.7
2021/12,164.8,291.7
2022/1,169.8,309.9
2022/2,174.9,292.7


In [27]:
# Result Dictionary
result = {}
result['use'] = []
result['not'] = []

# 1) 계절 필터링
df_all_day_use_spring = df_all_day_use[(df_all_day_use.ym == '2021/5') | (df_all_day_use.ym == '2022/3') | (df_all_day_use.ym == '2022/4')]
df_all_day_not_spring = df_all_day_not[(df_all_day_not.ym == '2021/5') | (df_all_day_not.ym == '2022/3') | (df_all_day_not.ym == '2022/4')]
    
df_all_day_use_summer = df_all_day_use[(df_all_day_use.ym == '2021/6') | (df_all_day_use.ym == '2021/7') | (df_all_day_use.ym == '2021/8')]
df_all_day_not_summer = df_all_day_not[(df_all_day_not.ym == '2021/6') | (df_all_day_not.ym == '2021/7') | (df_all_day_not.ym == '2021/8')]
    
df_all_day_use_autumn = df_all_day_use[(df_all_day_use.ym == '2021/9') | (df_all_day_use.ym == '2021/10') | (df_all_day_use.ym == '2021/11')]
df_all_day_not_autumn = df_all_day_not[(df_all_day_not.ym == '2021/9') | (df_all_day_not.ym == '2021/10') | (df_all_day_not.ym == '2021/11')]
    
df_all_day_use_winter = df_all_day_use[(df_all_day_use.ym == '2021/12') | (df_all_day_use.ym == '2022/1') | (df_all_day_use.ym == '2022/2')]
df_all_day_not_winter = df_all_day_not[(df_all_day_not.ym == '2021/12') | (df_all_day_not.ym == '2022/1') | (df_all_day_not.ym == '2022/2')]
    
# 2) 태양광 사용/미사용 가구별 값 합산
use_consum_spring = np.sum(df_all_day_use_spring['전력소비량(kWh)'])
not_consum_spring = np.sum(df_all_day_not_spring['전력소비량(kWh)'])
    
use_consum_summer = np.sum(df_all_day_use_summer['전력소비량(kWh)'])
not_consum_summer = np.sum(df_all_day_not_summer['전력소비량(kWh)'])
    
use_consum_autumn = np.sum(df_all_day_use_autumn['전력소비량(kWh)'])
not_consum_autumn = np.sum(df_all_day_not_autumn['전력소비량(kWh)'])
    
use_consum_winter = np.sum(df_all_day_use_winter['전력소비량(kWh)'])
not_consum_winter = np.sum(df_all_day_not_winter['전력소비량(kWh)'])
    
# 3) 평균 구하기
use_consum_spring_mean = round(use_consum_spring / (len(df_all_day_use_spring.owner.unique())*3), 1)
not_consum_spring_mean = round(not_consum_spring / (len(df_all_day_not_spring.owner.unique())*3), 1)

use_consum_summer_mean = round(use_consum_summer / (len(df_all_day_use_summer.owner.unique())*3), 1)
not_consum_summer_mean = round(not_consum_summer / (len(df_all_day_not_summer.owner.unique())*3), 1)
    
use_consum_autumn_mean = round(use_consum_autumn / (len(df_all_day_use_autumn.owner.unique())*3), 1)
not_consum_autumn_mean = round(not_consum_autumn / (len(df_all_day_not_autumn.owner.unique())*3), 1)
    
use_consum_winter_mean = round(use_consum_winter / (len(df_all_day_use_winter.owner.unique())*3), 1)
not_consum_winter_mean = round(not_consum_winter / (len(df_all_day_not_winter.owner.unique())*3), 1)
    
# 4) 값 넣기
result['use'] = [use_consum_spring_mean, use_consum_summer_mean, use_consum_autumn_mean, use_consum_winter_mean]
result['not'] = [not_consum_spring_mean, not_consum_summer_mean, not_consum_autumn_mean, not_consum_winter_mean]

result = pd.DataFrame(result)
result.index = ['spring', 'summer', 'autumn', 'winter']
result

,use,not
spring,200.9,239.2
summer,242.2,296.1
autumn,174.9,256.1
winter,169.8,298.1


In [28]:
# Result Dictionary
result = {}
result['use'] = []
result['not'] = []

# 1) 1년 필터링
df_all_day_use_year = df_all_day_use[(df_all_day_use.ym == '2021/5') | (df_all_day_use.ym == '2021/6') | (df_all_day_use.ym == '2021/7') |
                                    (df_all_day_use.ym == '2021/8') | (df_all_day_use.ym == '2021/9') | (df_all_day_use.ym == '2021/10') |
                                    (df_all_day_use.ym == '2021/11') | (df_all_day_use.ym == '2021/12') | (df_all_day_use.ym == '2022/1') |
                                    (df_all_day_use.ym == '2022/2') | (df_all_day_use.ym == '2022/3') | (df_all_day_use.ym == '2022/4')]
df_all_day_not_year = df_all_day_not[(df_all_day_not.ym == '2021/5') | (df_all_day_not.ym == '2021/6') | (df_all_day_not.ym == '2021/7') |
                                    (df_all_day_not.ym == '2021/8') | (df_all_day_not.ym == '2021/9') | (df_all_day_not.ym == '2021/10') |
                                    (df_all_day_not.ym == '2021/11') | (df_all_day_not.ym == '2021/12') | (df_all_day_not.ym == '2022/1') |
                                    (df_all_day_not.ym == '2022/2') | (df_all_day_not.ym == '2022/3') | (df_all_day_not.ym == '2022/4')]
    
# 2) 태양광 사용/미사용 가구별 값 합산
use_consum_year = np.sum(df_all_day_use_year['전력소비량(kWh)'])
not_consum_year = np.sum(df_all_day_not_year['전력소비량(kWh)'])
    
# 3) 평균 구하기
use_consum_year_mean = round(use_consum_year / (len(df_all_day_use_year.owner.unique())*12), 1)
not_consum_year_mean = round(not_consum_year / (len(df_all_day_not_year.owner.unique())*12), 1)
    
# 4) 값 넣기
result['use'] = use_consum_year_mean
result['not'] = not_consum_year_mean

result

{'use': 197.0, 'not': 272.4}

<hr>

### **2. 월간/계절별/1년 평균 가구당 전력 생산량(발전량)**
- 태양광 사용 가구

In [29]:
# Result Dictionary
result = {}
result['발전량(kWh)'] = []

# Fill Dictionary
date_list = ['2021/5', '2021/6', '2021/7', '2021/8', '2021/9', '2021/10', '2021/11', '2021/12',
             '2022/1', '2022/2', '2022/3', '2022/4']

for i in range(len(date_list)):
    # 1) Year/Month 필터링
    df_all_day_use_f = df_all_day_use[df_all_day_use.ym == date_list[i]]
    
    # 2) 태양광 사용/미사용 가구별 값 합산
    use_consum = np.sum(df_all_day_use_f['발전량(kWh)'])
    
    # 3) 평균 구하기
    use_consum_mean = round(use_consum / len(df_all_day_use_f.owner.unique()), 1)

    # 4) 값 넣기
    result['발전량(kWh)'].append(use_consum_mean)
    
result = pd.DataFrame(result)
result.index = date_list
result

,발전량(kWh)
2021/5,1898.3
2021/6,1813.6
2021/7,1819.2
2021/8,1541.9
2021/9,1311.0
2021/10,1633.1
2021/11,1387.3
2021/12,1449.0
2022/1,1569.1
2022/2,1726.0


In [30]:
# Result Dictionary
result = {}
result['발전량(kWh)'] = []

# 1) 계절 필터링
df_all_day_use_spring = df_all_day_use[(df_all_day_use.ym == '2021/5') | (df_all_day_use.ym == '2022/3') | (df_all_day_use.ym == '2022/4')]
df_all_day_use_summer = df_all_day_use[(df_all_day_use.ym == '2021/6') | (df_all_day_use.ym == '2021/7') | (df_all_day_use.ym == '2021/8')]
df_all_day_use_autumn = df_all_day_use[(df_all_day_use.ym == '2021/9') | (df_all_day_use.ym == '2021/10') | (df_all_day_use.ym == '2021/11')]
df_all_day_use_winter = df_all_day_use[(df_all_day_use.ym == '2021/12') | (df_all_day_use.ym == '2022/1') | (df_all_day_use.ym == '2022/2')]

# 2) 태양광 사용/미사용 가구별 값 합산
use_consum_spring = np.sum(df_all_day_use_spring['발전량(kWh)'])
use_consum_summer = np.sum(df_all_day_use_summer['발전량(kWh)'])
use_consum_autumn = np.sum(df_all_day_use_autumn['발전량(kWh)'])
use_consum_winter = np.sum(df_all_day_use_winter['발전량(kWh)'])
    
# 3) 평균 구하기
use_consum_spring_mean = round(use_consum_spring / (len(df_all_day_use_spring.owner.unique())*3), 1)
use_consum_summer_mean = round(use_consum_summer / (len(df_all_day_use_summer.owner.unique())*3), 1)
use_consum_autumn_mean = round(use_consum_autumn / (len(df_all_day_use_autumn.owner.unique())*3), 1)
use_consum_winter_mean = round(use_consum_winter / (len(df_all_day_use_winter.owner.unique())*3), 1)
    
# 4) 값 넣기
result['발전량(kWh)'] = [use_consum_spring_mean, use_consum_summer_mean, use_consum_autumn_mean, use_consum_winter_mean]

result = pd.DataFrame(result)
result.index = ['spring', 'summer', 'autumn', 'winter']
result

,발전량(kWh)
spring,1877.2
summer,1724.9
autumn,1443.8
winter,1581.4


In [31]:
# Result Dictionary
result = {}
result['발전량(kWh)'] = []

# 1) 1년 필터링
df_all_day_use_year = df_all_day_use[(df_all_day_use.ym == '2021/5') | (df_all_day_use.ym == '2021/6') | (df_all_day_use.ym == '2021/7') |
                                    (df_all_day_use.ym == '2021/8') | (df_all_day_use.ym == '2021/9') | (df_all_day_use.ym == '2021/10') |
                                    (df_all_day_use.ym == '2021/11') | (df_all_day_use.ym == '2021/12') | (df_all_day_use.ym == '2022/1') |
                                    (df_all_day_use.ym == '2022/2') | (df_all_day_use.ym == '2022/3') | (df_all_day_use.ym == '2022/4')]

# 2) 태양광 사용/미사용 가구별 값 합산
use_consum_year = np.sum(df_all_day_use_year['발전량(kWh)'])

# 3) 평균 구하기
use_consum_year_mean = round(use_consum_year / (len(df_all_day_use_year.owner.unique())*12), 1)

# 4) 값 넣기
result['발전량(kWh)'] = use_consum_year_mean

result

{'발전량(kWh)': 1656.8}

<hr>

### **3. 월간/계절별/1년 평균 가구당 전력 수출량(잉여전력량)**
- 태양광 사용 가구

In [32]:
# Result Dictionary
result = {}
result['잉여전력량(kWh)'] = []
result['잉여전력량/발전량'] = []

# Fill Dictionary
date_list = ['2021/5', '2021/6', '2021/7', '2021/8', '2021/9', '2021/10', '2021/11', '2021/12',
             '2022/1', '2022/2', '2022/3', '2022/4']

for i in range(len(date_list)):
    # 1) Year/Month 필터링
    df_all_day_use_f = df_all_day_use[df_all_day_use.ym == date_list[i]]
    
    # 2) 태양광 사용/미사용 가구별 값 합산
    use_export = np.sum(df_all_day_use_f['잉여전력량(kWh)'])
    use_percent = np.sum(df_all_day_use_f['잉여전력량/발전량'])
    
    # 3) 평균 구하기
    use_export_mean = round(use_export / len(df_all_day_use_f.owner.unique()), 1)
    use_percent_mean = round(use_percent / len(df_all_day_use_f.owner.unique()), 1)

    # 4) 값 넣기
    result['잉여전력량(kWh)'].append(use_export_mean)
    result['잉여전력량/발전량'].append(use_percent_mean)
    
result = pd.DataFrame(result)
result.index = date_list
result

,잉여전력량(kWh),잉여전력량/발전량
2021/5,1729.1,27.9
2021/6,1664.8,27.4
2021/7,1610.7,27.2
2021/8,1359.1,26.9
2021/9,1179.1,26.7
2021/10,1492.2,28.1
2021/11,1262.8,26.7
2021/12,1331.3,28.4
2022/1,1446.0,28.5
2022/2,1590.3,25.8


In [33]:
# Result Dictionary
result = {}
result['잉여전력량(kWh)'] = []
result['잉여전력량/발전량'] = []

# 1) 계절 필터링
df_all_day_use_spring = df_all_day_use[(df_all_day_use.ym == '2021/5') | (df_all_day_use.ym == '2022/3') | (df_all_day_use.ym == '2022/4')]
df_all_day_use_summer = df_all_day_use[(df_all_day_use.ym == '2021/6') | (df_all_day_use.ym == '2021/7') | (df_all_day_use.ym == '2021/8')]
df_all_day_use_autumn = df_all_day_use[(df_all_day_use.ym == '2021/9') | (df_all_day_use.ym == '2021/10') | (df_all_day_use.ym == '2021/11')]
df_all_day_use_winter = df_all_day_use[(df_all_day_use.ym == '2021/12') | (df_all_day_use.ym == '2022/1') | (df_all_day_use.ym == '2022/2')]

# 2) 태양광 사용/미사용 가구별 값 합산
use_consum_spring = np.sum(df_all_day_use_spring['잉여전력량(kWh)'])
use_percent_spring = np.sum(df_all_day_use_spring['잉여전력량/발전량'])

use_consum_summer = np.sum(df_all_day_use_summer['잉여전력량(kWh)'])
use_percent_summer = np.sum(df_all_day_use_summer['잉여전력량/발전량'])

use_consum_autumn = np.sum(df_all_day_use_autumn['잉여전력량(kWh)'])
use_percent_autumn = np.sum(df_all_day_use_autumn['잉여전력량/발전량'])

use_consum_winter = np.sum(df_all_day_use_winter['잉여전력량(kWh)'])
use_percent_winter = np.sum(df_all_day_use_winter['잉여전력량/발전량'])
    
# 3) 평균 구하기
use_consum_spring_mean = round(use_consum_spring / (len(df_all_day_use_spring.owner.unique())*3), 1)
use_percent_spring_mean = round(use_percent_spring / (len(df_all_day_use_spring.owner.unique())*3), 1)

use_consum_summer_mean = round(use_consum_summer / (len(df_all_day_use_summer.owner.unique())*3), 1)
use_percent_summer_mean = round(use_percent_summer / (len(df_all_day_use_summer.owner.unique())*3), 1)

use_consum_autumn_mean = round(use_consum_autumn / (len(df_all_day_use_autumn.owner.unique())*3), 1)
use_percent_autumng_mean = round(use_percent_autumn / (len(df_all_day_use_autumn.owner.unique())*3), 1)

use_consum_winter_mean = round(use_consum_winter / (len(df_all_day_use_winter.owner.unique())*3), 1)
use_percent_winter_mean = round(use_percent_winter / (len(df_all_day_use_winter.owner.unique())*3), 1)

# 4) 값 넣기
result['잉여전력량(kWh)'] = [use_consum_spring_mean, use_consum_summer_mean, use_consum_autumn_mean, use_consum_winter_mean]
result['잉여전력량/발전량'] = [use_percent_spring_mean, use_percent_summer_mean, use_percent_autumng_mean, use_percent_winter_mean]

result = pd.DataFrame(result)
result.index = ['spring', 'summer', 'autumn', 'winter']
result

,잉여전력량(kWh),잉여전력량/발전량
spring,1712.5,27.7
summer,1544.9,27.2
autumn,1311.3,27.2
winter,1455.9,27.6


In [34]:
# Result Dictionary
result = {}
result['잉여전력량(kWh)'] = []
result['잉여전력량/발전량'] = []

# 1) 1년 필터링
df_all_day_use_year = df_all_day_use[(df_all_day_use.ym == '2021/5') | (df_all_day_use.ym == '2021/6') | (df_all_day_use.ym == '2021/7') |
                                    (df_all_day_use.ym == '2021/8') | (df_all_day_use.ym == '2021/9') | (df_all_day_use.ym == '2021/10') |
                                    (df_all_day_use.ym == '2021/11') | (df_all_day_use.ym == '2021/12') | (df_all_day_use.ym == '2022/1') |
                                    (df_all_day_use.ym == '2022/2') | (df_all_day_use.ym == '2022/3') | (df_all_day_use.ym == '2022/4')]

# 2) 태양광 사용/미사용 가구별 값 합산
use_consum_year = np.sum(df_all_day_use_year['잉여전력량(kWh)'])
use_percent_year = np.sum(df_all_day_use_year['잉여전력량/발전량'])

# 3) 평균 구하기
use_consum_year_mean = round(use_consum_year / (len(df_all_day_use_year.owner.unique())*12), 1)
use_percent_year_mean = round(use_percent_year / (len(df_all_day_use_year.owner.unique())*12), 1)

# 4) 값 넣기
result['잉여전력량(kWh)'] = use_consum_year_mean
result['잉여전력량/발전량'] = use_percent_year_mean

result

{'잉여전력량(kWh)': 1506.1, '잉여전력량/발전량': 27.4}

<hr>

### **4. SSR, SCR, 설비이용률, 발전시간**
- 태양광 사용 가구

In [35]:
# Result Dictionary
result = {}
result['SSR'] = []
result['SCR'] = []
result['설비이용률'] = []
result['발전시간'] = []

# Fill Dictionary
date_list = ['2021/5', '2021/6', '2021/7', '2021/8', '2021/9', '2021/10', '2021/11', '2021/12',
             '2022/1', '2022/2', '2022/3', '2022/4']

for i in range(len(date_list)):
    # 1) Year/Month 필터링
    df_all_day_use_f = df_all_day_use[df_all_day_use.ym == date_list[i]]
    
    # 2) 태양광 사용/미사용 가구별 값 합산
    ssr = np.sum(df_all_day_use_f['자가공급률'])
    scr = np.sum(df_all_day_use_f['자가소비율'])
    use_rate = np.sum(df_all_day_use_f['이용률'])
    use_hour = np.sum(df_all_day_use_f['발전시간'])
    
    # 3) 평균 구하기
    ssr_mean = round(ssr / len(df_all_day_use_f), 2)
    scr_mean = round(scr / len(df_all_day_use_f), 2)
    use_rate_mean = round(use_rate / len(df_all_day_use_f), 2)
    use_hour_mean = round(use_hour / len(df_all_day_use_f), 2)
    
    # 4) 값 넣기
    result['SSR'].append(ssr_mean)
    result['SCR'].append(scr_mean)
    result['설비이용률'].append(use_rate_mean)
    result['발전시간'].append(use_hour_mean)
    
result = pd.DataFrame(result)
result.index = date_list
result

,SSR,SCR,설비이용률,발전시간
2021/5,0.79,0.10,14.18,3.40
2021/6,0.79,0.09,13.99,3.36
2021/7,0.73,0.12,13.58,3.26
2021/8,0.69,0.13,11.51,2.76
2021/9,0.75,0.11,10.12,2.43
2021/10,0.76,0.09,12.19,2.93
2021/11,0.71,0.11,10.70,2.57
2021/12,0.71,0.08,10.82,2.60
2022/1,0.72,0.08,11.72,2.81
2022/2,0.77,0.08,14.27,3.42


<hr>

### **5. 월간/계절별/1년 평균 가구당 전력 그리드 소비량**
- 태양광 사용 가구
- 태양광 미사용 가구

In [36]:
# Result Dictionary
result = {}
result['use'] = []
result['not'] = []

# Fill Dictionary
date_list = ['2021/5', '2021/6', '2021/7', '2021/8', '2021/9', '2021/10', '2021/11', '2021/12',
             '2022/1', '2022/2', '2022/3', '2022/4']

for i in range(len(date_list)):
    # 1) Year/Month 필터링
    df_all_day_use_f = df_all_day_use[df_all_day_use.ym == date_list[i]]
    df_all_day_not_f = df_all_day_not[df_all_day_not.ym == date_list[i]]
    
    # 2) 태양광 사용/미사용 가구별 값 합산
    use_consum = np.sum(df_all_day_use_f['수전전력량(kWh)'])
    not_consum = np.sum(df_all_day_not_f['수전전력량(kWh)'])
    
    # 3) 평균 구하기
    use_consum_mean = round(use_consum / len(df_all_day_use_f.owner.unique()), 1)
    not_consum_mean = round(not_consum / len(df_all_day_not_f.owner.unique()), 1)
    
    # 4) 값 넣기
    result['use'].append(use_consum_mean)
    result['not'].append(not_consum_mean)
    
result = pd.DataFrame(result)
result.index = date_list
result

,use,not
2021/5,37.5,238.2
2021/6,34.8,232.0
2021/7,76.1,336.2
2021/8,75.7,320.2
2021/9,40.6,267.8
2021/10,39.7,242.8
2021/11,47.1,257.7
2021/12,47.1,291.7
2022/1,46.7,309.9
2022/2,39.2,292.7


In [37]:
# Result Dictionary
result = {}
result['use'] = []
result['not'] = []

# 1) 계절 필터링
df_all_day_use_spring = df_all_day_use[(df_all_day_use.ym == '2021/5') | (df_all_day_use.ym == '2022/3') | (df_all_day_use.ym == '2022/4')]
df_all_day_not_spring = df_all_day_not[(df_all_day_not.ym == '2021/5') | (df_all_day_not.ym == '2022/3') | (df_all_day_not.ym == '2022/4')]
    
df_all_day_use_summer = df_all_day_use[(df_all_day_use.ym == '2021/6') | (df_all_day_use.ym == '2021/7') | (df_all_day_use.ym == '2021/8')]
df_all_day_not_summer = df_all_day_not[(df_all_day_not.ym == '2021/6') | (df_all_day_not.ym == '2021/7') | (df_all_day_not.ym == '2021/8')]
    
df_all_day_use_autumn = df_all_day_use[(df_all_day_use.ym == '2021/9') | (df_all_day_use.ym == '2021/10') | (df_all_day_use.ym == '2021/11')]
df_all_day_not_autumn = df_all_day_not[(df_all_day_not.ym == '2021/9') | (df_all_day_not.ym == '2021/10') | (df_all_day_not.ym == '2021/11')]
    
df_all_day_use_winter = df_all_day_use[(df_all_day_use.ym == '2021/12') | (df_all_day_use.ym == '2022/1') | (df_all_day_use.ym == '2022/2')]
df_all_day_not_winter = df_all_day_not[(df_all_day_not.ym == '2021/12') | (df_all_day_not.ym == '2022/1') | (df_all_day_not.ym == '2022/2')]
    
# 2) 태양광 사용/미사용 가구별 값 합산
use_consum_spring = np.sum(df_all_day_use_spring['수전전력량(kWh)'])
not_consum_spring = np.sum(df_all_day_not_spring['수전전력량(kWh)'])
    
use_consum_summer = np.sum(df_all_day_use_summer['수전전력량(kWh)'])
not_consum_summer = np.sum(df_all_day_not_summer['수전전력량(kWh)'])
    
use_consum_autumn = np.sum(df_all_day_use_autumn['수전전력량(kWh)'])
not_consum_autumn = np.sum(df_all_day_not_autumn['수전전력량(kWh)'])
    
use_consum_winter = np.sum(df_all_day_use_winter['수전전력량(kWh)'])
not_consum_winter = np.sum(df_all_day_not_winter['수전전력량(kWh)'])
    
# 3) 평균 구하기
use_consum_spring_mean = round(use_consum_spring / (len(df_all_day_use_spring.owner.unique())*3), 1)
not_consum_spring_mean = round(not_consum_spring / (len(df_all_day_not_spring.owner.unique())*3), 1)

use_consum_summer_mean = round(use_consum_summer / (len(df_all_day_use_summer.owner.unique())*3), 1)
not_consum_summer_mean = round(not_consum_summer / (len(df_all_day_not_summer.owner.unique())*3), 1)
    
use_consum_autumn_mean = round(use_consum_autumn / (len(df_all_day_use_autumn.owner.unique())*3), 1)
not_consum_autumn_mean = round(not_consum_autumn / (len(df_all_day_not_autumn.owner.unique())*3), 1)
    
use_consum_winter_mean = round(use_consum_winter / (len(df_all_day_use_winter.owner.unique())*3), 1)
not_consum_winter_mean = round(not_consum_winter / (len(df_all_day_not_winter.owner.unique())*3), 1)
    
# 4) 값 넣기
result['use'] = [use_consum_spring_mean, use_consum_summer_mean, use_consum_autumn_mean, use_consum_winter_mean]
result['not'] = [not_consum_spring_mean, not_consum_summer_mean, not_consum_autumn_mean, not_consum_winter_mean]

result = pd.DataFrame(result)
result.index = ['spring', 'summer', 'autumn', 'winter']
result

,use,not
spring,36.2,239.2
summer,62.2,296.1
autumn,42.5,256.1
winter,44.3,298.1


In [38]:
# Result Dictionary
result = {}
result['use'] = []
result['not'] = []

# 1) 1년 필터링
df_all_day_use_year = df_all_day_use[(df_all_day_use.ym == '2021/5') | (df_all_day_use.ym == '2021/6') | (df_all_day_use.ym == '2021/7') |
                                    (df_all_day_use.ym == '2021/8') | (df_all_day_use.ym == '2021/9') | (df_all_day_use.ym == '2021/10') |
                                    (df_all_day_use.ym == '2021/11') | (df_all_day_use.ym == '2021/12') | (df_all_day_use.ym == '2022/1') |
                                    (df_all_day_use.ym == '2022/2') | (df_all_day_use.ym == '2022/3') | (df_all_day_use.ym == '2022/4')]
df_all_day_not_year = df_all_day_not[(df_all_day_not.ym == '2021/5') | (df_all_day_not.ym == '2021/6') | (df_all_day_not.ym == '2021/7') |
                                    (df_all_day_not.ym == '2021/8') | (df_all_day_not.ym == '2021/9') | (df_all_day_not.ym == '2021/10') |
                                    (df_all_day_not.ym == '2021/11') | (df_all_day_not.ym == '2021/12') | (df_all_day_not.ym == '2022/1') |
                                    (df_all_day_not.ym == '2022/2') | (df_all_day_not.ym == '2022/3') | (df_all_day_not.ym == '2022/4')]
    
# 2) 태양광 사용/미사용 가구별 값 합산
use_consum_year = np.sum(df_all_day_use_year['수전전력량(kWh)'])
not_consum_year = np.sum(df_all_day_not_year['수전전력량(kWh)'])
    
# 3) 평균 구하기
use_consum_year_mean = round(use_consum_year / (len(df_all_day_use_year.owner.unique())*12), 1)
not_consum_year_mean = round(not_consum_year / (len(df_all_day_not_year.owner.unique())*12), 1)
    
# 4) 값 넣기
result['use'] = use_consum_year_mean
result['not'] = not_consum_year_mean

result

{'use': 46.3, 'not': 272.4}

<hr>

### **6. SCR(자가소비율)**
- 태양광 사용 가구
- 개인정보 포함으로 코드 삭제함.

<hr>

### **7. SSR(자가공급률)**
- 태양광 사용 가구
- 개인정보 포함으로 코드 삭제함.